### Japan microgrid - Generate result tables

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform

%matplotlib inline

In [ ]:
from rccp_utils import *

In [ ]:
gdrive_folder = '..'
print('gdrive_folder=', gdrive_folder)

In [ ]:
project_folder = '../doutorado/robusto/RCCP'
antoine_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(gdrive_folder, "rccp_experiments")
results_folder = os.path.join(gdrive_folder, "rccp_results")
cost_results_folder = os.path.join(output_folder, 'consolidated_results', 'df')
var_results_folder = os.path.join(output_folder, 'consolidated_results', 'df')
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)


### 1.9. Create the output folders for processed results

In [ ]:
reportfolder = os.path.join(output_folder, 'consolidated_results')
reportfolder_graph = os.path.join(reportfolder, 'graphs')
reportfolder_table = os.path.join(reportfolder, 'tables')
if not os.path.exists(reportfolder_graph):
    os.makedirs(reportfolder_graph)
if not os.path.exists(reportfolder_table):
    os.makedirs(reportfolder_table)
print('Saving files on folder: ' + reportfolder)

### Read processed results dataframe

In [ ]:
df_tables = pd.read_pickle(os.path.join(reportfolder, 'df_japan_table1.pkl.gz'))

In [ ]:
df_tables.head(2)

### 1. Table with average solution time (hours)

In [ ]:
df_time_list = []
for csv_file in glob.glob(os.path.join(output_folder, 'run_sim_japan_forecast_avg', 'ccp_model*.csv')):
    df_time_list.append(pd.read_csv(csv_file))
    print('Read file', csv_file)
df_time = pd.concat(df_time_list)
df_time.info()

In [ ]:
df_time['OriginalInstanceName'] = df_time['instance_name']
df_time['instance_name'] = df_time['instance_name'].str.replace('instance_deltamin10_', '', regex=False).str.replace('.txt', '', regex=False)
df_time['Γ'] = np.round(100 * df_time['model_parameters'].astype(str).replace('none', '0.0', regex=False).astype(float) / 48, 0).astype(int)  #.replace('none', '', regex=False)
df_time['-'] = 'Γ=' + df_time['Γ'].astype(str) + '%'
df_time['Instance'] = df_time['instance_name']
df_time['Model'] = df_time['model']
df_time['time_spent'] = np.round(df_time['time_spent'], 2)
df_time.loc[(df_time['model'] == 'deterministic'), '-'] = ''
df_time.head(10)

In [ ]:
time_table = pd.pivot_table(df_time, values='time_spent', index=['Instance'],
                    columns=['Model', '-'], aggfunc=np.sum).reindex([('deterministic', ''),('robust-budget', 'Γ=0%'), ('robust-budget', 'Γ=20%'),('robust-budget', 'Γ=40%'),
                                                                    ('robust-budget', 'Γ=60%'), ('robust-budget', 'Γ=80%'), ('robust-budget', 'Γ=100%')], axis=1)
time_table.to_html(os.path.join(reportfolder_table, 'japan-time-table.html'))
time_table

### 2. Complete Table with all results for Japan Microgrid

In [ ]:
df_complete = df_tables.copy()
df_complete['Γ'] = df_complete['Gamma']
df_complete['Γ'] = 'Γ=' + df_complete['Γ'].astype(str) + '%'
df_complete.loc[(df_complete['Model'] == 'deterministic'), 'Γ'] = '-'
df_complete = df_complete.set_index(['Instance', 'Model', 'Γ', 'RTCS Policy'])
df_complete = df_complete[['Cost Avg', 'Cost Std', 'Cost CVaR', 'OC Cost Avg', 'Penalty Freq', 'Renewables Util Avg', 'SOC Avg']] 
df_complete.to_html(os.path.join(reportfolder_table, 'japan-complete-table.html'))
df_complete

### 3. Summary Table with best results for Japan Microgrid

In [ ]:
# Get the best RTCS_Policy for each instance and deterministic model
df_best = df_complete.reset_index()
df_best_det_cvar = df_best[(df_best['Model'] == 'deterministic')].groupby(by=['Instance', 'Model', 'Γ']).min()[['Cost CVaR']]
print('Best CVaR (smallest CVaR) for each instance, for deterministic model')
display(df_best_det_cvar)
df_best_det_policy = df_best[(df_best['Model'] == 'deterministic')].merge(df_best_det_cvar, on=['Instance', 'Model', 'Γ', 'Cost CVaR'])
print('Best RTCS_Policy (with smallest CVaR) for each instance, for deterministic model')
display(df_best_det_policy)
df_best_det_policy = df_best_det_policy.iloc[[0, 3, 4, 5]]
print('Best RTCS_Policy (with smallest CVaR) for each instance, for deterministic model => after tie break')
display(df_best_det_policy)
# Join the best deterministic RTCS_Policy with all remaining models
df_summary = df_complete.reset_index().merge(df_best_det_policy, on=['Instance', 'RTCS Policy'], suffixes=('', '_y'))
print('Deterministic vs. Robust Models')
#display(df_summary)
df_summary = df_summary[['Instance', 'Model', 'Γ', 'Cost Avg', 'Cost Std', 'Cost CVaR', 'OC Cost Avg', 'Penalty Freq', 'Renewables Util Avg', 'SOC Avg']]
print('Results for all Instances')
display(df_summary)
for season in df_best['Instance'].unique():
    print('Results for Instance=', season)
    df_table_season = df_summary[(df_summary['Instance'] == season)]
    df_table_season = df_table_season.set_index(['Instance', 'Model', 'Γ']).T
    df_table_season = df_table_season.reindex([(season, 'deterministic', '-'),(season, 'robust-budget', 'Γ=0%'), (season, 'robust-budget', 'Γ=20%'),(season, 'robust-budget', 'Γ=40%'),
                                                (season, 'robust-budget', 'Γ=60%'), (season, 'robust-budget', 'Γ=80%'), (season, 'robust-budget', 'Γ=100%')], axis=1)
    display(df_table_season)
    df_table_season.to_html(os.path.join(reportfolder_table, 'japan-summary-table-{}.html'.format(season)))